In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import imblearn                                      ### sampling the data using the smote method
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
import sklearn
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from xgboost import cv
import numpy as np

In [2]:
df = pd.read_csv('preprocessing_approach_1.csv')

In [33]:
#df_1 = pd.read_csv('preprocessing_approach_1_test.csv')

In [3]:
X = df.drop(columns=['y'])
y = df['y']

In [34]:
# np.random.seed(seed=351)
# X_train_norm = df.drop(columns=['y'])
# Y_train = df['y']
# X_val_norm = df_1.drop(columns=['y'])
# Y_val = df_1['y']

In [4]:
np.random.seed(seed=42)
X_train_norm, X_val_norm, Y_train, Y_val = train_test_split(X, y, test_size=0.1)     ### splitting the dataset into training and test set

In [5]:
def sampling(over_under,X,y):
    if over_under == 'under':
        sampler = imblearn.under_sampling.RandomUnderSampler()
        X_under, y_under = sampler.fit_resample(X, y)
        return X_under,y_under
    elif over_under == 'over':
        ros = RandomOverSampler(random_state=rs)
        X_over, y_over = ros.fit_resample(X, y)
        return X_over,y_over
    else:
        sampler = SMOTE()
        X_smote, y_smote = sampler.fit_resample(X, y)
        return X_smote,y_smote
X_final,y_final = sampling('smote',X_train_norm,Y_train)

In [158]:
clf_svc = sklearn.svm.SVC(C = 50, degree = 1, gamma = "auto", kernel = "rbf", probability = True)

# Initializing Multi-layer perceptron  classifier
clf_mlp = MLPClassifier(activation = "relu", alpha = 0.1, hidden_layer_sizes = (10,10,10),
                            learning_rate = "constant", max_iter = 2000, random_state = 1000)

# Initialing Nu Support Vector classifier
clf_nusvc = sklearn.svm.NuSVC(degree = 1, kernel = "rbf", nu = 0.25, probability = True)

# Initializing Random Forest classifier
clf_rfc = sklearn.ensemble.RandomForestClassifier(n_estimators = 500, criterion = "gini", max_depth = 10,
                                     max_features = "auto", min_samples_leaf = 0.005,
                                     min_samples_split = 0.005, n_jobs = -1, random_state = 1000)
lr = sklearn.linear_model.LogisticRegression(multi_class='multinomial', solver='saga', penalty = 'elasticnet', C = 1, l1_ratio = 0.5)  # defining meta-classifier


xgb = XGBClassifier(n_estimators = 200, subsample=0.8, max_depth=50, eta=0.5, gamma=1, min_child_weight=10, alpha=1)   
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=474, oob_score = True, criterion='gini',max_depth=90, min_samples_split=10, max_features='auto', min_samples_leaf = 6)

clf_stack = StackingClassifier(classifiers =[clf_svc, clf_mlp, clf_rfc, clf_nusvc], meta_classifier = lr, use_features_in_secondary = True)
clf_stack = clf_stack.fit(X_final, y_final)
kfold = StratifiedKFold(n_splits=10)
results = cross_val_score(clf_stack, X_final, y_final, cv=10)
print("sklearn:::", results.mean())
print(f"Training accuracy is {clf_stack.score(X_final, y_final)}")
print(f"Testing accuracy is {clf_stack.score(X_val_norm, Y_val)}")

/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: T

sklearn::: 0.5375661375661377
Training accuracy is 0.9927536231884058
Testing accuracy is 0.2962962962962963


In [1]:
from mlxtend.classifier import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.neural_network import MLPClassifier
clf_stack = None
lr = sklearn.linear_model.LogisticRegression(multi_class='multinomial', solver='saga', penalty = 'elasticnet', C = 1, l1_ratio = 0.5)  # defining meta-classifier
SVM = svm.SVC(kernel = 'linear')
xgb = XGBClassifier(n_estimators = 200, subsample=0.8, max_depth=50, eta=0.5, gamma=1, min_child_weight=10, alpha=1)   
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=474, oob_score = True, criterion='gini',max_depth=90, min_samples_split=10, max_features='auto', min_samples_leaf = 6)
mlp = MLPClassifier(activation = "relu", alpha = 0.1, hidden_layer_sizes = (10,10,10), learning_rate = "constant", max_iter = 2000, random_state = 1000)
NB = GaussianNB() 
knn = KNeighborsClassifier(n_neighbors=10)
clf_stack = StackingClassifier(classifiers =[rf, xgb,SVM, NB], meta_classifier = lr, use_probas = True)
clf_stack_1 = clf_stack.fit(X_final, y_final)
#trans = clf_stack.fit_transform(X_final, y_final) clf_stack.predict_meta_features
trans = clf_stack.predict_meta_features(X_final)
print("Transforms=",trans, trans.shape)
kfold = StratifiedKFold(n_splits=10)
results = cross_val_score(clf_stack_1, X_final, y_final, cv=10)
print("sklearn:::", results.mean())
print(f"Training accuracy is {clf_stack_1.score(X_final, y_final)}")
print(f"Testing accuracy is {clf_stack_1.score(X_val_norm, Y_val)}")

NameError: name 'sklearn' is not defined

In [36]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
model5 = None
def logistic_regress(X_train_norm, X_test, Y_train,Y_test):
    global model5
    model5 = sklearn.linear_model.LogisticRegression(multi_class='multinomial', solver='saga', penalty = 'elasticnet', C = 1, l1_ratio = 0.5)
    model5.fit(X_train_norm, Y_train)
    kfold = StratifiedKFold(n_splits=10)
    results = cross_val_score(model5, X_train_norm, Y_train, cv=10)
    print("sklearn:::", results.mean())
    print(f"Training accuracy is {model5.score(X_train_norm, Y_train)}")
    print(f"Testing accuracy is {model5.score(X_test, Y_test)}")
logistic_regress(X_final, X_val_norm, y_final, Y_val)

/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: T

sklearn::: 0.7363636363636364
Training accuracy is 0.9
Testing accuracy is 0.6428571428571429


/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
## from sklearn import svm
from sklearn.model_selection import KFold
model_svm = None
def svm1(X_train_norm, X_test, Y_train,Y_test):
    global model_svm
    model_svm = svm.SVC(kernel = 'linear')
    model_svm.fit(X_train_norm, Y_train)
    kfold = StratifiedKFold(n_splits=10)
    results = cross_val_score(model_svm, X_train_norm, Y_train, cv=10)
    print("sklearn:::", results.mean())
    print(f"Training accuracy is {model_svm.score(X_train_norm, Y_train)}")
    print(f"Testing accuracy is {model_svm.score(X_test, Y_test)}")
    
svm1(X_final, X_val_norm, y_final, Y_val)

In [80]:
model4 = None
def xgboost(X_train_norm, X_test,Y_train, Y_test):
    global model4
    model4 = XGBClassifier(n_estimators = 200, subsample=0.8, max_depth=50, eta=0.5, gamma=1, min_child_weight=10, alpha=1)

    model4.fit(X_train_norm, Y_train)
    kfold = StratifiedKFold(n_splits=10)
    results = cross_val_score(model4, X_train_norm, Y_train, cv=10)
    print("sklearn:::", results, results.mean())
    print(f"Training accuracy is {model4.score(X_train_norm, Y_train)}")
    print(f"Testing accuracy is {model4.score(X_test, Y_test)}")
xgboost(X_final, X_val_norm, y_final, Y_val)

/ext3/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:28:44] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/ext3/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:28:44] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/ext3/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[08:28:48] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/ext3/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:28:49] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/ext3/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:28:49] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
sklearn::: [0.73684211 0.65789474 0.73684211 0.81578947 0.81578947 0.78947368
 0.67567568 0.83783784 0.78378378 0.72972973] 0.7579658605974396
Training accuracy is 0.9840425531914894
Testing accuracy is 0.6666666666666666


In [46]:
model3 = None
def random_forest(X_train_norm, X_test, Y_train,Y_test):
    global model3
    model3 = sklearn.ensemble.RandomForestClassifier(n_estimators=474, oob_score = True, criterion='gini',max_depth=90, min_samples_split=10, max_features='auto', min_samples_leaf = 6)
    model3.fit(X_train_norm, Y_train)
    print(model3.oob_score_)
    print(f"Training accuracy is {model3.score(X_train_norm, Y_train)}")
    print(f"Testing accuracy is {model3.score(X_test, Y_test)}")
random_forest(X_final, X_val_norm, y_final, Y_val)

/ext3/miniconda3/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


0.6127819548872181
Training accuracy is 0.9360902255639098
Testing accuracy is 0.6666666666666666
